<a href="https://colab.research.google.com/github/NSPFD/PROJECTWORK-OF-WOMANIUM/blob/main/Hamiltonian_simulation_project_of_WOMANIUM_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HAMILTONIAN SIMULATION BASED ON VQE ALGORITHIMS**

Quantum Hamiltonian simulation is one of the most important problems in quantum computing. It consists in modeling the time evolution of a quantum system, ruled by some not natively executable Hamiltonian in a specific device. Simulations of this type are essential to comprehend quantum systems that are described by a complex dynamics, such as molecules in chemistry or materials in condensed matter physics, where classical computers fail due the exponential scale up required by larger quantum systems.

In this guide, you will be using Classiq to work with simple problems of Hamiltonian simulation using two different methods:

Trotter-Suzuki decomposition

Quantum Hamiltonian simulation has several significant applications. Some of them are particularly interesting for physicists:

Quantum Chemistry: It helps in predicting properties of molecules using quantum methods such as the Quantum Variational Eigensolver (VQE), which can be more efficiently than classical methods in certain conditions.
Materials Science: It aids in designing new materials with desired properties by simulating their quantum mechanical behavior.
Optimization and Machine Learning: Quantum simulations can be used to solve complex optimization problems and improve machine learning algorithms.
Mathematically, we can approach the Hamiltonian Simulation problem as the following: Suppose we have a quantum computer that can run natively the evolution operators of the set of Hamiltonians {Hj}. Now, suppose we want to understand the dynamics of the following, more complex, Hamiltonian:

What are the operations that this quantum computer can effectively execute?
Hfull=∑jhjHj,

For some real values hj. Therefore, we would need some kind of decomposition from the operator of Eq. (1) into our set of operations, formed by {e−itHj}, where e−itHj is the evolution operator of the hamiltonian H.

Trotter-Suzuki decomposition
Now that the problem is stated, a possible solution is the Trotter-Suzuki decomposition, also known as Trotterization. Simply put, this method involves "breaking" the evolution operator into the evolution operators of its components. In the first order, it looks like this:

e−itH=exp{−it∑jhjHj}≈(∏je−it,hj,Hj/n)n+O(t2/n).

More details about this decomposition can be found in [1].

This formula is very important and has applications in several scenarios. Additionally, if we need an error that scales better than t2, it is possible to achieve higher-order Trotter-Suzuki formulas. For example, the second order Trotter-Suzuki would look like this:

e−it(H1+H2)=exp{−i(t1H1+t2H2)}≈(e−it1H1/(2n)e−it2H2/ne−it1H1/(2n))n+O(t3/n).

Higher orders for the Trotter-Suzuki formula can be found in [1]. However, as the number of terms in the hamiltonian and the approximation order increases, it gets harder to construct the Trotter-Suzuki formulas. For this end, we can use Classiq's function suzuki_trotter() to execute a Trotter decomposition of an operator, specifying the order and number of repetitions. Lets take a look on how does it work with an example:

Now that the inputs for the Trotter-Suzuki method are defined, lets apply it to the simple example of approximating the dynamics of the following two-qubits hamiltonian

H=0.3Z⊗Z+0.7X⊗I+0.2I⊗X,

which represents two spins interacting in a transversal field. First, we need to identify the operators {Hj} and its respective coefficients {hj}:

H1=Z⊗Z, coefficient: h1=0.3;H2=X⊗I, coefficient: h2=0.7;H3=I⊗X, coefficient: h3=0.2.

The second step is to define the number n of repetitions into which we will "break" the evolution. In this case, we will apply it for n=10. The order of the Trotter-Suzuki decomposition can be altered in the parameter order. In the current case we will be applying it to first order.


[ ]


In [ ]:
from classiq import *

#Defining the Hamiltonian:
HAMILTONIAN = [PauliTerm(pauli=[Pauli.Z, Pauli.Z], coefficient=0.3),
            PauliTerm(pauli=[Pauli.X, Pauli.I], coefficient=0.7),
            PauliTerm(pauli=[Pauli.I, Pauli.X], coefficient=0.2),]
@qfunc
def main(qba: Output[QArray[QBit]]):
    allocate(2, qba)
    suzuki_trotter(
        HAMILTONIAN,
        evolution_coefficient=1.0,
        order=1,
        repetitions=10,
        qbv=qba,
    )


qmod = create_model(main)
qprog = synthesize(qmod)
write_qmod(qmod, "trotter")
show(qprog)

Exponentiation with depth constraint

Exponentiation with depth constraint
It is also possible to generate an efficient decomposition of an evolution operator with Classiq's function exponentiation_with_depth_constraint. Given the maximum depth of the decomposition, and the inputs related to the Hamiltonian itself, the synthesizer finds the most accurate higher order Trotter decomposition and applies it. For the example above, considering a maximum depth of  N=30  the code would be:

In [ ]:
from classiq import *

@qfunc
def main(qba: Output[QArray[QBit]]):

    allocate(2, qba)
    exponentiation_with_depth_constraint(
        HAMILTONIAN,
        evolution_coefficient=1,
        max_depth=30,
        qbv=qba,
    )


qmod = create_model(main)
write_qmod(qmod, "exponentiation")
qprog = synthesize(qmod)
show(qprog)